# Pronalaženje motiva

Pomoćne funkcije sa prethodnog časa

In [1]:
'''Pomoćna funkcija koja pretvara broj u simbol nukleotida'''

def number_to_symbol(number):
    mapping = {
        0: 'A',
        1: 'T',
        2: 'C',
        3: 'G'
    }
    
    if number not in [0,1,2,3]:
        print(f'Invalid number: {number}')
        return None
    
    return mapping[number]

In [2]:
'''Pomoćna funkcija koja pretvara simbol nukleotida u broj'''

def symbol_to_number(symbol):
    mapping = {
        'A': 0,
        'T': 1,
        'C': 2,
        'G': 3
    }
    
    return mapping[symbol.upper()]

In [3]:
'''Pomoćna funkcija koja transformiše broj u k-mer'''

def number_to_pattern(number, k):
    if k == 1:
        return number_to_symbol(number)
    r = number % 4
    q = number // 4
    last_symbol = number_to_symbol(r)
    
    return number_to_pattern(q, k - 1) + last_symbol

In [4]:
'''Pomoćna funkcija koja izračunava Hamingovo rastojanje između sekvenci iste dužine'''

def hamming_distance(sequence_1, sequence_2):
    n = len(sequence_1)
    distance = 0
    
    for i in range(n):
        if sequence_1[i] != sequence_2[i]:
            distance += 1
            
    return distance

Nove funkcije

In [5]:
'''
Funkcija računa udaljenost šablonske sekvence od DNK sekvence,
pronalazeći poravnanje sekvenci koje rezultuje najmanjim Hamingovim rastojanjem
'''

def d(pattern, dna):
    k = len(pattern)
    total_distance = 0
    
    for sequence in dna:
        n = len(sequence)
        min_distance = float('inf')
        
        for i in range(0, n - k + 1):
            current_pattern = sequence[i : i + k]
            current_distance = hamming_distance(pattern, current_pattern.upper())
            
            if current_distance < min_distance:
                min_distance = current_distance
                
        total_distance += min_distance
    return total_distance

In [6]:
# Primer
dna = [
    'ttaccttAAC',
    'gATAtctgtc',
    'ACGgcgttcg',
    'ccctAAAgag',
    'cgtcAGAggt'
]

pattern ='AAA'
d(pattern, dna)

5

*Median string*
algoritam pronalazi sekvencu dužine $k$ koja predstavlja motiv na osnovu najveće ukupnog najmanjeg rastojanja (u kontekstu funkcije `d`) u odnosu na sve DNK sekvence

In [7]:
'''Algoritam za pronalaženje mediana niske kao kandidata za sekvencu motiva'''

def median_string(dna, k):
    distance = float('inf')
    median_pattern = None
    
    for i in range(4 ** k):
        pattern = number_to_pattern(i, k)
        current_distance = d(pattern, dna)
        if current_distance < distance:
            distance = current_distance
            median_pattern = pattern

    return median_pattern

In [8]:
# Primer
dna = [
    'ttaccttAAC',
    'gATAtctgtc',
    'ACGgcgttcg',
    'ccctAAAgag',
    'cgtcAGAggt'
]
k = 3
median_string(dna, k)

'CCT'

In [9]:
'''Konstrukcija count matrice koja prebrojava nukleotide u skupu motiva prema pozicijama'''

def count(motifs, k, pseudocount=0):
    counts = [[pseudocount] * 4 for _ in range(k)]    
    
    for motif in motifs:
        for i in range(k):
            current_nuc = motif[i]
            j = symbol_to_number(current_nuc)
            counts[i][j] += 1
            
    return counts

In [10]:
# Primer
motifs = [
    'ATGCAA',
    'CTCCAA',
    'AGCCAA',
]
k = 6
count(motifs, k)

[[2, 0, 1, 0],
 [0, 2, 0, 1],
 [0, 0, 2, 1],
 [0, 0, 3, 0],
 [3, 0, 0, 0],
 [3, 0, 0, 0]]

In [11]:
'''Računanje skora skupa motiva (manji skor = veća sličnost između motiva)'''

def score(motifs, k):
    total_score = 0
    counts = count(motifs, k)
            
    for i in range(k):
        most_frequent = counts[i].index(max(counts[i]))
        for j in range(4):
            if j != most_frequent:
                 total_score += counts[i][j]
    
    return total_score

In [12]:
# Primer
motifs = [
    'ATGCAA',
    'CTCCAA',
    'AGCCAA',
]
k = 6
count(motifs, k)

[[2, 0, 1, 0],
 [0, 2, 0, 1],
 [0, 0, 2, 1],
 [0, 0, 3, 0],
 [3, 0, 0, 0],
 [3, 0, 0, 0]]

In [13]:
'''
Konstrukcija profilne matrice čije vrednosti predstavljaju 
verovatnoće pojave nukleotida na pojedinačnim pozicijama u motivu
'''

def profile(motifs, k):
    t = len(motifs)
    pseudocount = 3
    counts = count(motifs, k, pseudocount)
    profile_matrix = [[0] * 4 for _ in range(k)]
    
    for i in range(k):
        for j in range(4):
            profile_matrix[i][j] = counts[i][j] / (t + (4 * pseudocount))
            
    return profile_matrix

In [14]:
# Primer
motifs = [
    'ATGCAA',
    'CTCCAA',
    'AGCCAA',
]
k = 6
profile(motifs, k)

[[0.3333333333333333, 0.2, 0.26666666666666666, 0.2],
 [0.2, 0.3333333333333333, 0.2, 0.26666666666666666],
 [0.2, 0.2, 0.3333333333333333, 0.26666666666666666],
 [0.2, 0.2, 0.4, 0.2],
 [0.4, 0.2, 0.2, 0.2],
 [0.4, 0.2, 0.2, 0.2]]

In [15]:
'''
Funkcija računa verovatnoću sekvence u odnosu na profil skupa motiva.
Verovatnoća sekvence predstavlja proizvod verovatnoće pojave nukleotida
na pojedinačnim pozicijama motiva - pretpostavka nezavisnosti
'''

def probability(motif_profile, pattern, k):
    prob = 1
    for i in range(k):
        symbol = pattern[i]
        j = symbol_to_number(symbol)
        symbol_prob = motif_profile[i][j]
        prob *= symbol_prob
    return prob

In [16]:
# Primer
motifs = [
    'ATGCAA',
    'CTCCAA',
    'AGCCAA',
]
k = 6
motif_profile = profile(motifs, k)
pattern = 'ATCCTA'
probability(motif_profile, pattern, k)

0.0011851851851851854

In [17]:
'''
Pronalaženje najverovatnije podniske dužine k u DNK sekvenci
u odnosu na profilnu matricu motiva (u kontekstu funkcije probability)
'''

def most_probable_k_mer(motif_profile, sequence, k):
    n = len(sequence)
    max_prob = float('-inf')
    max_pattern = None
    
    for i in range(n - k + 1):
        pattern = sequence[i : i + k]
        prob = probability(motif_profile, pattern, k)
        if prob > max_prob:
            max_prob = prob
            max_pattern = pattern
            
    return max_pattern

In [18]:
# Primer
motifs = [
    'ATGCAA',
    'CTCCAA',
    'AGCCAA',
]
k = 6
motif_profile = profile(motifs, k)
sequence = 'TTGTAGGAAACATCCTACCCAGGAAT'
most_probable_k_mer(motif_profile, sequence, k)

'ATCCTA'

*Greedy motif search* algoritam pronalazi motive u DNK sekvencama pohlepnim pristupom, konstruišući matricu motiva dodavanjem sledećeg motiva koji predstavlja najverovatniju podnisku dužine k u i-toj sekvenci u odnosu na do tada sastavljen skup motiva.

In [19]:
'''Pronalaženje motiva pomoću greedy motif search algoritma'''
def greedy_motif_search(dna, k, t):
    best_motifs = [sequence[:k] for sequence in dna]
    best_score = score(best_motifs, k)

    first_sequence = dna[0]
    for i in range(len(first_sequence) - k + 1):
        motif_1 = first_sequence[i : i + k]
        motifs = [motif_1]
        
        for j in range(1, t):
            motif_profile = profile(motifs, k)
            motif_i = most_probable_k_mer(motif_profile, dna[j], k)
            motifs.append(motif_i)
            
        current_score = score(motifs, k)
        if current_score < best_score:
            best_score = current_score
            best_motifs = motifs
            
    return best_motifs

In [20]:
# Primer
dna = [
    'ttACCTtaac',
    'gATGTctgtc',
    'acgGCGTtag',
    'ccctaACGAg',
    'cgtcagAGGT'
]

k = 4
greedy_motif_search(dna, k, len(dna))

['ACCT', 'ATGT', 'acgG', 'ACGA', 'AGGT']

In [21]:
'''Izdvajanje skupa motiva dužine k iz skupa DNK sekvenci'''

def motifs_from_profile(motif_profile, dna, k):
    motifs = []
    for sequence in dna:
        motif_i = most_probable_k_mer(motif_profile, sequence, k)
        motifs.append(motif_i)
    return motifs

*Randomized motif search* algoritam pronalazi motive primenom EM (expectation maximization) algoritma. Algoritam polazi od nasumično odabranih motiva a zatim konstruiše profilnu matricu na osnovu skupa motiva koju koristi za konstruisanje nove matrice motiva.
Iterativni postupak približava skup motiva traženom skupu motiva, ali je moguće zaglavljivanje u lokalnom optimumu. Iz tog razloga, algoritam se pokreće više puta i odabira najbolji rezultat

In [22]:
'''Pronalaženje motiva pomoću randomized motif search algoritma'''

import random

def randomized_motif_search(dna, k, t):
    motifs = []
    for sequence in dna:
        n = len(sequence)
        i = random.randrange(0, n - k + 1)
        motifs.append(sequence[i : i + k])
        
    best_motifs = motifs
    best_score = score(best_motifs, k)
    
    while True:
        motif_profile = profile(motifs, k)
        motifs = motifs_from_profile(motif_profile, dna, k)
        
        current_score = score(motifs, k)
        if current_score < best_score:
            best_score = current_score
            best_motifs = motifs
        else:
            return best_score, best_motifs

In [23]:
# Primer
dna = [
    'ttACCTtaac',
    'gATGTctgtc',
    'acgGCGTtag',
    'ccctaACGAg',
    'cgtcagAGGT'
]

k = 4
min_score = float('inf')
min_motifs = None

for _ in range(100):
    best_score, best_motifs = randomized_motif_search(dna, k, len(dna))
    if best_score <= min_score:
        min_score = best_score
        min_motifs = best_motifs
        
print(min_motifs)

['ACCT', 'ATGT', 'acgG', 'ACGA', 'AGGT']


In [24]:
'''Pronalaženje najverovatnijeg šablona dužine k u DNK sekvenci 
na osnovu Gibsove raspodele predstavljene profilnom matricom skupa motiva
'''

def most_probable_gibbs_k_mer(motif_profile, sequence, k):
    probabilities = []
    patterns = []
    
    n = len(sequence)
    prob_sum = 0
    
    for i in range(n - k + 1):
        pattern = sequence[i : i + k]
        prob = probability(motif_profile, pattern, k)
        prob_sum += prob
        
        patterns.append(pattern)
        probabilities.append(prob)
        
    random_pos = random.random() * prob_sum
    
    current_sum = 0
    for i in range(n - k + 1):
        prob = probabilities[i]
        current_sum += prob
        
        if current_sum >= random_pos:
            return patterns[i]

*Gibbs sampler* algoritam pronalazi motive na sličan način kao i randomized motif search uz bitnu razliku da se ne odabiraju uvek najsličniji šabloni već se na osnovu Gibsove raspodele daje verovatnoća izbora i ostalim šablonima.

In [25]:
'''Pronalaženje motiva pomoću randomized Gibbs sampler algoritma'''

from copy import deepcopy
def gibbs_sampler(dna, k, t, N):
    motifs = []
    for sequence in dna:
        n = len(sequence)
        i = random.randrange(0, n - k + 1)
        motifs.append(sequence[i : i + k])
        
    best_motifs = deepcopy(motifs)
    best_score = score(best_motifs, k)
    
    for _ in range(N):
        i = random.randrange(0, t)
        del motifs[i]
        motif_profile = profile(motifs, k)
        motif_i = most_probable_gibbs_k_mer(motif_profile, dna[i], k)
        motifs.insert(i, motif_i)
        
        current_score = score(motifs, k)
        if current_score < best_score:
            best_score = current_score
            best_motifs = deepcopy(motifs)
            
    return best_score, best_motifs

In [26]:
# Primer
dna = [
    'ttACCTtaac',
    'gATGTctgtc',
    'acgGCGTtag',
    'ccctaACGAg',
    'cgtcagAGGT'
]

k = 4
min_score = float('inf')
min_motifs = None
N = 100


for _ in range(100):
    best_score, best_motifs = gibbs_sampler(dna, k, len(dna), N)
    if best_score <= min_score:
        min_score = best_score
        min_motifs = best_motifs
        
print(min_motifs)

['ACCT', 'ATGT', 'acgG', 'ACGA', 'AGGT']
